# Install, Paths and Parameters

In [1]:
# Requirements. Need to restart after installation (it sucks, I am sorry haha)
!pip install torch==1.7.1
!pip install torchvision==0.8.2
!pip install torchattacks

     |████████████████████████████████| 776.8 MB 16 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.11.1+cu111 requires torch==1.10.0, but you have torch 1.7.1 which is incompatible.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.7.1 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.7.1 which is incompatible.
     |████████████████████████████████| 12.8 MB 5.2 MB/s 
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.11.1+cu111
    Uninstalling torchvision-0.11.1+cu111:
      Successfully uninstalled torchvision-0.11.1+cu111
     |████████████████████████████████| 102 kB 3.9 MB/s 


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import pandas as pd
import numpy as np
import json
import torch
from typing import List, Callable
import random

# seed
SEED = 42
random.seed(SEED)
torch.manual_seed(SEED)
np.random.seed(SEED)

In [3]:
# general path
DRIVE_PATH = '/content/drive/MyDrive/'
DL_PROJECT_PATH = os.path.join(DRIVE_PATH, 'DeepLearningProject')
ADV_PATH = os.path.join(DL_PROJECT_PATH, 'AdversarialAttacks')

# filenames and label path
ORG_LABEL_PATH = os.path.join(ADV_PATH,'ori_data/correct_labels_cl.txt')

# image paths
ORIGINAL_IMAGES_PATH = os.path.join(ADV_PATH,'ori_data/ImageNetClasses/')

In [4]:
def get_random_classes(number_of_classes: int = 50, min_rand_class: int = 0, max_rand_class: int = 999):
  return np.random.randint(low=min_rand_class, high=max_rand_class, size=(number_of_classes,))

def get_random_indexes(number_of_images: int = 50000, n_samples=1000):
  return np.random.choice(50000, 1000, replace=False)

INDEX_SUBSET = get_random_indexes()

# Import DINO
Official repo: https://github.com/facebookresearch/dino

In [5]:
# Load pretrained weights from PyTorch
device = 'cuda'
vits16 = torch.hub.load('facebookresearch/dino:main', 'dino_vits8').to(device)

Using cache found in /root/.cache/torch/hub/facebookresearch_dino_main


In [6]:
# Hyperparameters
N_LAST_BLOCKS = 4 # Took from official repo
PATCH_SIZE=8

In [7]:
# Define and load pretrained weights for linear classifier on ImageNet
from torch import nn
class LinearClassifier(nn.Module):
    """Linear layer to train on top of frozen features"""
    def __init__(self, dim, num_labels=1000):
        super(LinearClassifier, self).__init__()
        self.num_labels = num_labels
        self.linear = nn.Linear(dim, num_labels) 
        self.linear.weight.data.normal_(mean=0.0, std=0.01)
        self.linear.bias.data.zero_()

    def forward(self, x):
        # flatten
        x = x.view(x.size(0), -1)

        # linear layer
        return self.linear(x)

linear_classifier = LinearClassifier(vits16.embed_dim * N_LAST_BLOCKS, num_labels=1000)
linear_classifier = linear_classifier.cuda()

linear_state_dict = torch.hub.load_state_dict_from_url(url="https://dl.fbaipublicfiles.com/dino/" + "dino_deitsmall8_pretrain/dino_deitsmall8_linearweights.pth")["state_dict"]

# Update state dict to avoid crash. Workaround.
linear_state_dict['linear.weight'] = linear_state_dict.pop('module.linear.weight')
linear_state_dict['linear.bias'] = linear_state_dict.pop('module.linear.bias')

# Load pre-trained weights
linear_classifier.load_state_dict(linear_state_dict, strict=True)

<All keys matched successfully>

# Load data

In [8]:
# Custom loader
from torch.utils.data import Dataset
from torchvision.io import read_image
import traceback
from PIL import Image

class ImageDataset(Dataset):
  def __init__(self, img_folder: str, file_name: str, transform: callable, class_subset: List[int] = None, index_subset: List[int] = None):
    super().__init__()
    self.transform=transform
    self.img_folder=img_folder
    self.data = self.create_df(file_name)
    self.class_subset = class_subset
    if self.class_subset is None:
      if index_subset is not None:
          self.data_subset = self.data.iloc[index_subset]
      else:
        self.data_subset = self.data
    else:
      self.data_subset = self.data[self.data['label'].isin(self.class_subset)] 
  
  def create_df(self, file_name: str):
    df = pd.read_csv(file_name, sep=" ", header=None)
    df.columns=['file', 'label']
    return df
    
  def __len__(self):
    return len(self.data_subset)
  
  def __getitem__(self, index):
    img = Image.open(os.path.join(self.img_folder,self.data_subset['file'].iloc[index]))
    img = img.convert('RGB')

    img=self.transform(img)
    target=self.data_subset['label'].iloc[index]

    return img,target,self.data_subset['file'].iloc[index]


In [9]:
from torchvision import transforms as pth_transforms

BATCH_SIZE = 8

# Create loader
# Taken from official repo: https://github.com/facebookresearch/dino/blob/main/eval_linear.py
loader_transform = pth_transforms.Compose([
    pth_transforms.Resize(256, interpolation=3),
    pth_transforms.CenterCrop(224),
    pth_transforms.ToTensor()
])

org_dataset = ImageDataset(img_folder = ORIGINAL_IMAGES_PATH,
                           file_name = ORG_LABEL_PATH,
                           transform=loader_transform,
                           index_subset=INDEX_SUBSET)

org_loader = torch.utils.data.DataLoader(
    org_dataset,
    batch_size=BATCH_SIZE,
    num_workers=1,
    pin_memory=True,
)

In [10]:
BATCH_SIZE

8

In [11]:
class ViTModel(torch.nn.Module):
    def __init__(self, vits16, linear_layer):
        """
        In the constructor we instantiate two nn.Linear modules and assign them as
        member variables.
        """
        super(ViTModel, self).__init__()
        self.vits16=vits16
        self.linear_layer=linear_layer
        self.transform = transform = pth_transforms.Compose([
            pth_transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
        ])
        
        self.vits16.eval()
        self.linear_layer.eval()

    def forward(self, x, grad=True):
        """
        In the forward function we accept a Tensor of input data and we must return
        a Tensor of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Tensors.
        """
        if grad is False:
          with torch.no_grad():
            x = self.transform(x)
            
            # forward
            intermediate_output = self.vits16.get_intermediate_layers(x, 4)
            output = torch.cat([x[:, 0] for x in intermediate_output], dim=-1)
            output = linear_classifier(output)
            return output
        else:
          x = self.transform(x)
            
          # forward
          intermediate_output = self.vits16.get_intermediate_layers(x, 4)
          output = torch.cat([x[:, 0] for x in intermediate_output], dim=-1)
          output = linear_classifier(output)
          return output

In [12]:
model = ViTModel(vits16, linear_classifier)

# Attack model

In [20]:
import torchattacks
from torchattacks import *

attacks = [
    FGSM(model, eps=0.062),
    CW(model, c=50, lr=0.00155, steps=30)
]

In [21]:

for atk in attacks :
    
    print("-"*70)
    print(atk)
    
    correct = 0
    clean_correct = 0
    total = 0
    
    for images, labels, _ in tqdm.tqdm(org_loader):
        
        start = time.time()

        cuda_images = images.to(device)
        clean_outputs = model(cuda_images, False)
        del cuda_images

        adv_images = atk(images, labels)
        labels = labels.to(device)
        outputs = model(adv_images)

        _, pre = torch.max(outputs.data, 1)
        _, pre_clean = torch.max(clean_outputs.data, 1)

        total += len(labels.flatten())
        correct += (pre == labels).sum()
        clean_correct += (pre_clean == labels).sum()

        del images
        del labels
        torch.cuda.empty_cache()

    print('Total elapsed time (sec): %.2f' % (time.time() - start))
    print('Accuracy against attack: %.2f %%' % (100 * float(correct) / total))
    print('Clean accuracy: %.2f %%' % (100 * float(clean_correct) / total))

----------------------------------------------------------------------
FGSM(model_name=ViTModel, device=cuda:0, eps=0.062, attack_mode=default, return_type=float)


100%|██████████| 125/125 [04:02<00:00,  1.94s/it]


Total elapsed time (sec): 1.98
Accuracy against attack: 1.10 %
Clean accuracy: 79.10 %
----------------------------------------------------------------------
CW(model_name=ViTModel, device=cuda:0, c=50, kappa=0, steps=30, lr=0.00155, attack_mode=default, return_type=float)


100%|██████████| 125/125 [31:50<00:00, 15.28s/it]

Total elapsed time (sec): 12.15
Accuracy against attack: 0.00 %
Clean accuracy: 79.10 %


#### Outputs
* Clean accuracy: 79.10%
* FGWS accuracy: 1.10%
* CW accuracy: 0.00%